# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Dec 12 2022 12:23PM,252837,21,418543289,"NBTY Global, Inc.",Released
1,Dec 12 2022 12:14PM,252834,10,0086242014,ISDIN Corporation,Released
2,Dec 12 2022 12:14PM,252834,10,0086242012,ISDIN Corporation,Released
3,Dec 12 2022 12:14PM,252834,10,0086242013,ISDIN Corporation,Released
4,Dec 12 2022 12:14PM,252834,10,0086242036,ISDIN Corporation,Released
5,Dec 12 2022 12:14PM,252834,10,0086242035,ISDIN Corporation,Released
6,Dec 12 2022 12:14PM,252834,10,0086242034,ISDIN Corporation,Released
7,Dec 12 2022 12:14PM,252834,10,0086242038,ISDIN Corporation,Released
8,Dec 12 2022 12:14PM,252834,10,0086242054,ISDIN Corporation,Released
9,Dec 12 2022 12:14PM,252834,10,0086242058,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
30,252833,Released,23
31,252834,Released,37
32,252835,Released,1
33,252836,Released,1
34,252837,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
252833,NaN,23.0
252834,NaN,37.0
252835,NaN,1.0
252836,NaN,1.0
252837,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
252777,0.0,1.0
252778,0.0,1.0
252783,1.0,0.0
252784,0.0,1.0
252785,0.0,8.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
252777,0,1
252778,0,1
252783,1,0
252784,0,1
252785,0,8


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,252777,0,1
1,252778,0,1
2,252783,1,0
3,252784,0,1
4,252785,0,8


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,252777,,1
1,252778,,1
2,252783,1,
3,252784,,1
4,252785,,8


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Dec 12 2022 12:23PM,252837,21,"NBTY Global, Inc."
1,Dec 12 2022 12:14PM,252834,10,ISDIN Corporation
38,Dec 12 2022 12:10PM,252833,19,"Innogenix, LLC"
61,Dec 12 2022 12:09PM,252836,16,Sartorius Bioprocess Solutions
62,Dec 12 2022 12:09PM,252835,10,Yusen – 3D Matrix
63,Dec 12 2022 11:59AM,252832,10,ISDIN Corporation
68,Dec 12 2022 11:55AM,252831,19,"GCH Granules USA, Inc."
69,Dec 12 2022 11:40AM,252830,10,Eywa Pharma Inc.
70,Dec 12 2022 11:35AM,252829,12,Doctors Scientific Organica
71,Dec 12 2022 11:34AM,252825,10,Eywa Pharma Inc.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Dec 12 2022 12:23PM,252837,21,"NBTY Global, Inc.",,1
1,Dec 12 2022 12:14PM,252834,10,ISDIN Corporation,,37
2,Dec 12 2022 12:10PM,252833,19,"Innogenix, LLC",,23
3,Dec 12 2022 12:09PM,252836,16,Sartorius Bioprocess Solutions,,1
4,Dec 12 2022 12:09PM,252835,10,Yusen – 3D Matrix,,1
5,Dec 12 2022 11:59AM,252832,10,ISDIN Corporation,,5
6,Dec 12 2022 11:55AM,252831,19,"GCH Granules USA, Inc.",,1
7,Dec 12 2022 11:40AM,252830,10,Eywa Pharma Inc.,,1
8,Dec 12 2022 11:35AM,252829,12,Doctors Scientific Organica,,1
9,Dec 12 2022 11:34AM,252825,10,Eywa Pharma Inc.,,4


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 12 2022 12:23PM,252837,21,"NBTY Global, Inc.",1,
1,Dec 12 2022 12:14PM,252834,10,ISDIN Corporation,37,
2,Dec 12 2022 12:10PM,252833,19,"Innogenix, LLC",23,
3,Dec 12 2022 12:09PM,252836,16,Sartorius Bioprocess Solutions,1,
4,Dec 12 2022 12:09PM,252835,10,Yusen – 3D Matrix,1,
5,Dec 12 2022 11:59AM,252832,10,ISDIN Corporation,5,
6,Dec 12 2022 11:55AM,252831,19,"GCH Granules USA, Inc.",1,
7,Dec 12 2022 11:40AM,252830,10,Eywa Pharma Inc.,1,
8,Dec 12 2022 11:35AM,252829,12,Doctors Scientific Organica,1,
9,Dec 12 2022 11:34AM,252825,10,Eywa Pharma Inc.,4,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 12 2022 12:23PM,252837,21,"NBTY Global, Inc.",1,
1,Dec 12 2022 12:14PM,252834,10,ISDIN Corporation,37,
2,Dec 12 2022 12:10PM,252833,19,"Innogenix, LLC",23,
3,Dec 12 2022 12:09PM,252836,16,Sartorius Bioprocess Solutions,1,
4,Dec 12 2022 12:09PM,252835,10,Yusen – 3D Matrix,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Dec 12 2022 12:23PM,252837,21,"NBTY Global, Inc.",1.0,NaN
1,Dec 12 2022 12:14PM,252834,10,ISDIN Corporation,37.0,NaN
2,Dec 12 2022 12:10PM,252833,19,"Innogenix, LLC",23.0,NaN
3,Dec 12 2022 12:09PM,252836,16,Sartorius Bioprocess Solutions,1.0,NaN
4,Dec 12 2022 12:09PM,252835,10,Yusen – 3D Matrix,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 12 2022 12:23PM,252837,21,"NBTY Global, Inc.",1.0,0.0
1,Dec 12 2022 12:14PM,252834,10,ISDIN Corporation,37.0,0.0
2,Dec 12 2022 12:10PM,252833,19,"Innogenix, LLC",23.0,0.0
3,Dec 12 2022 12:09PM,252836,16,Sartorius Bioprocess Solutions,1.0,0.0
4,Dec 12 2022 12:09PM,252835,10,Yusen – 3D Matrix,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,4297841,198.0,14.0
12,758401,2.0,1.0
16,252836,1.0,0.0
19,1769726,29.0,0.0
20,758341,3.0,0.0
21,758484,3.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,4297841,198.0,14.0
1,12,758401,2.0,1.0
2,16,252836,1.0,0.0
3,19,1769726,29.0,0.0
4,20,758341,3.0,0.0
5,21,758484,3.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,198.0,14.0
1,12,2.0,1.0
2,16,1.0,0.0
3,19,29.0,0.0
4,20,3.0,0.0
5,21,3.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,198.0
1,12,Released,2.0
2,16,Released,1.0
3,19,Released,29.0
4,20,Released,3.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,19,20,21
Status,,,,,,
Executing,14.0,1.0,0.0,0.0,0.0,0.0
Released,198.0,2.0,1.0,29.0,3.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,19,20,21
0,Executing,14.0,1.0,0.0,0.0,0.0,0.0
1,Released,198.0,2.0,1.0,29.0,3.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,19,20,21
0,Executing,14.0,1.0,0.0,0.0,0.0,0.0
1,Released,198.0,2.0,1.0,29.0,3.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()